In [ ]:
!pip install azure-storage-blob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from itertools import chain
import requests
import json
import re
import os

queries_for_ids = {
    "athletes_list": "https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes",
    "games_list": "https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{}/types/{}/weeks/{}/events?limit=100",
    "weeks_list": "https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{}/types/{}/weeks"
}

def unwrap_reference(obj, unwrap_elements=False):
  url = ""
  if (isinstance(obj, dict)):
    url = obj["$ref"]
  elif (isinstance(obj, str)):
    url = obj
  else:
    raise Exception("unsupported input type")
  obj = requests.get(url).json()
  result_objs = []
  if 'pageIndex' in obj.keys():
    if (obj["pageCount"] == 0):
      return []
    if (obj["pageCount"] == 1):
      if (unwrap_elements):
        for i, _ in enumerate(obj["items"]):
          obj["items"][i] = requests.get(obj["items"][i]["$ref"]).json()
      return obj["items"]
    for i in range(obj['pageCount']):
      page_objs = requests.get(url, params={"page": i + 1, "limit": 1000}).json()['items']
      if (unwrap_elements):
        for i, _ in enumerate(page_objs):
          page_objs[i] = requests.get(page_objs[i]["$ref"]).json()
      result_objs.extend(page_objs)
  else:
    return obj
  return result_objs

def extract_ids_from_url(url):
  matches = re.finditer(r"([^\/?]+)", url)
  ids = {}
  current_id = ""
  for matchNum, match in enumerate(matches):
    if (matchNum < 6 or "=" in match.group()):
      continue
    try:
      ids[current_id+"_id"] = int(match.group())
    except:
      current_id = match.group()
  return ids

def statistics_unwrap(obj):
  categories = {}
  for category in obj:
    stats = {}
    cat_name = category["name"]
    for stat in category["stats"]:
      stat = {"name": stat["name"], "value": stat["value"]}
      stats[stat["name"]] = stat
    del category["stats"]
    category.update(stats)
    categories[cat_name] = category
  return categories

## Function for games

In [ ]:
def load_game(event):
  base = event["competitions"][0]["$ref"]
  game = requests.get(base).json() # object with game info

  # teams
  teams = [unwrap_reference(competitor) for competitor in game["competitors"]]

  # team
  for team in teams:
    # do not unwrap team section
    if ("score" in team):
      team["score"] = unwrap_reference(team["score"]) # overall team score in the game
    if ("linescores" in team):
      team["linescores"] = unwrap_reference(team["linescores"])  # scores by quarters
    if ("roster" in team):
      team["roster"] = unwrap_reference(team["roster"])["entries"] # list of team with players statistics
        #statistics for players
      for player in team["roster"]:
        if ("statistics" in player):
          player["statistics"] = unwrap_reference(player["statistics"])["splits"]["categories"]
          player["statistics"] = statistics_unwrap(player["statistics"])

    if ("statistics" in team):
      team["statistics"] = unwrap_reference(team["statistics"])["splits"]["categories"]  # team stats
    if ("record" in team):
      team["record"] = unwrap_reference(team["record"]) # I dont know what is it, some statistics


  game["competitors"] = teams

  # other game stats
  if ("details" in game):
    game["details"] = unwrap_reference(game["details"]) # game moments
  if ("leaders" in game):
    game["leaders"] = unwrap_reference(game["leaders"]) # game leaders by categroies
  if ("officials" in game):
    game["officials"] = unwrap_reference(game["officials"]) # ???
  if ("drives" in game):
    game["drives"] = unwrap_reference(game["drives"]) # some play events i dont know

  # delete trash
  bad_sections = set(["situation","broadcasts",\
                "status", "links",\
                "type", "necessary", \
                "timeValid", \
                "neutralSite", \
                "divisionCompetition", \
                "conferenceCompetition",\
                "onWatchESPN", "recent", "ranks"])

  for section in list(game.keys()):
    if (section in bad_sections):
      del game[section]
    if (section.find("Available") != -1):
      del game[section]
    if (section.find("Source") != -1):
      del game[section]
  if ("venue" in game):
    game["venue"].pop("images", None)

  #save json
  game_id = game["id"]
  # json.dump(game, open(f"games/game_{game_id}.json", "w"))
  return game
game = load_game(requests.get("http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/271014026?lang=en&region=us").json())

In [ ]:
game

In [ ]:
game["competitors"][0]["roster"][0]["statistics"]

In [ ]:
pd.set_option("display.max_columns", None)
pd.json_normalize(game["competitors"], record_path="roster", record_prefix=".")

,.playerId,.period,.active,.starter,.forPlayerId,.valid,.didNotPlay,.athlete.$ref,.position.$ref,.statistics.general.name,.statistics.general.displayName,.statistics.general.shortDisplayName,.statistics.general.abbreviation,.statistics.general.summary,.statistics.general.fumblesForced.name,.statistics.general.fumblesForced.value,.statistics.general.fumblesRecovered.name,.statistics.general.fumblesRecovered.value,.statistics.general.fumblesRecoveredYards.name,.statistics.general.fumblesRecoveredYards.value,.statistics.general.fumblesTouchdowns.name,.statistics.general.fumblesTouchdowns.value,.statistics.general.gamesPlayed.name,.statistics.general.gamesPlayed.value,.statistics.general.defensiveFumblesTouchdowns.name,.statistics.general.defensiveFumblesTouchdowns.value,.statistics.defensive.name,.statistics.defensive.displayName,.statistics.defensive.shortDisplayName,.statistics.defensive.abbreviation,.statistics.defensive.summary,.statistics.defensive.assistTackles.name,.statistics.defensive.assistTackles.value,.statistics.defensive.avgInterceptionYards.name,.statistics.defensive.avgInterceptionYards.value,.statistics.defensive.avgSackYards.name,.statistics.defensive.avgSackYards.value,.statistics.defensive.avgStuffYards.name,.statistics.defensive.avgStuffYards.value,.statistics.defensive.blockedFieldGoalTouchdowns.name,.statistics.defensive.blockedFieldGoalTouchdowns.value,.statistics.defensive.blockedPuntTouchdowns.name,.statistics.defensive.blockedPuntTouchdowns.value,.statistics.defensive.defensiveTouchdowns.name,.statistics.defensive.defensiveTouchdowns.value,.statistics.defensive.hurries.name,.statistics.defensive.hurries.value,.statistics.defensive.kicksBlocked.name,.statistics.defensive.kicksBlocked.value,.statistics.defensive.longInterception.name,.statistics.defensive.longInterception.value,.statistics.defensive.miscTouchdowns.name,.statistics.defensive.miscTouchdowns.value,.statistics.defensive.passesBattedDown.name,.statistics.defensive.passesBattedDown.value,.statistics.defensive.passesDefended.name,.statistics.defensive.passesDefended.value,.statistics.defensive.QBHits.name,.statistics.defensive.QBHits.value,.statistics.defensive.twoPtReturns.name,.statistics.defensive.twoPtReturns.value,.statistics.defensive.sacks.name,.statistics.defensive.sacks.value,.statistics.defensive.sacksAssisted.name,.statistics.defensive.sacksAssisted.value,.statistics.defensive.sacksUnassisted.name,.statistics.defensive.sacksUnassisted.value,.statistics.defensive.sackYards.name,.statistics.defensive.sackYards.value,.statistics.defensive.safeties.name,.statistics.defensive.safeties.value,.statistics.defensive.soloTackles.name,.statistics.defensive.soloTackles.value,.statistics.defensive.stuffs.name,.statistics.defensive.stuffs.value,.statistics.defensive.stuffYards.name,.statistics.defensive.stuffYards.value,.statistics.defensive.tacklesForLoss.name,.statistics.defensive.tacklesForLoss.value,.statistics.defensive.tacklesYardsLost.name,.statistics.defensive.tacklesYardsLost.value,.statistics.defensive.teamGamesPlayed.name,.statistics.defensive.teamGamesPlayed.value,.statistics.defensive.totalTackles.name,.statistics.defensive.totalTackles.value,.statistics.defensive.yardsAllowed.name,.statistics.defensive.yardsAllowed.value,.statistics.defensive.pointsAllowed.name,.statistics.defensive.pointsAllowed.value,.statistics.defensive.onePtSafetiesMade.name,.statistics.defensive.onePtSafetiesMade.value,.statistics.defensiveInterceptions.name,.statistics.defensiveInterceptions.displayName,.statistics.defensiveInterceptions.shortDisplayName,.statistics.defensiveInterceptions.abbreviation,.statistics.defensiveInterceptions.summary,.statistics.defensiveInterceptions.interceptions.name,.statistics.defensiveInterceptions.interceptions.value,.statistics.defensiveInterceptions.interceptionTouchdowns.name,.statistics.defensiveInterceptions.interceptionTouchdowns.value,.statistics.defensiveInterceptions.interceptionYards.name,.statistics.defensiveIntercep

## Athlet


In [ ]:
url = "http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2022/athletes/3915511?lang=en&region=us"
athlet = requests.get(url).json()

In [ ]:
def preprocess_athlet(athlet):
  athlet["position"]["parent"] = unwrap_reference(athlet["position"]["parent"])
  athlet["contracts"] = unwrap_reference(athlet["contracts"])

  # college info
  if ("collegeAthlete" in athlet and "college" in athlet):
    athlet["college"] = unwrap_reference(athlet["college"])
    athlet["collegeAthlete"] = unwrap_reference(athlet["collegeAthlete"])
    college_athlet = athlet["collegeAthlete"]
    if ("statisticslog" in college_athlet):
      college_athlet["statisticslog"] = unwrap_reference(college_athlet["statisticslog"])
      seasons = college_athlet["statisticslog"]["entries"]
      stats_seasons = []
      for season in seasons:
        stats = unwrap_reference(season["statistics"][0]["statistics"])["splits"]["categories"]
        stats_seasons.append({"season": season["season"]["$ref"]})
        stats_seasons[-1]["statistics"] = statistics_unwrap(stats)

    athlet["collegeAthlete"] = {
        "statistics": stats_seasons,
        "team": college_athlet["team"],
        "experience": college_athlet["experience"],
        "position": college_athlet["position"],
        "college": college_athlet["college"]
    }

  # overall stats
  if ("statistics" in athlet):
    athlet["statistics"] = unwrap_reference(athlet["statistics"])
    stats = athlet["statistics"]["splits"]["categories"]
    athlet["statistics"] = statistics_unwrap(stats)

  # games played
  if ("eventLog" in athlet):
    athlet["eventLog"] = requests.get(athlet["eventLog"]["$ref"]+"&limit=1000").json()
    if ("events" in athlet):
      athlet["eventLog"] = athlet["eventLog"]["events"]["items"]
      athlet["eventLog"] = [event["competition"] for event in athlet["eventLog"]]
    else:
      athlet["eventLog"] = []

  bad_cols = ["alternateIds", "links", "headshot"]
  for section in bad_cols:
    try:
      del athlet[section]
    except:
      pass

  json.dump(athlet, open(f"athlets/athlet_{athlet['id']}.json", "w"))
  return athlet

processed_athlet = preprocess_athlet(athlet.copy())

## Team

In [ ]:
def preprocess_team(team):
  # record
  if ("record" in team):
    team["record"] = unwrap_reference(team["record"])
    team["record"] = statistics_unwrap(team["record"])

  if ("statistics" in team):
    team["statistics"] = unwrap_reference(team["statistics"])["splits"]["categories"]
    team["statistics"] = statistics_unwrap(team["statistics"])

  if ("leaders" in team):
    team["leaders"] = unwrap_reference(team["leaders"])
    leaders = [] # or split on categories?
    for category in team["leaders"]["categories"]:
      for athlete in category["leaders"]:
        leaders.append({"category": category["name"], "athlete_id": extract_ids_from_url(athlete["athlete"]["$ref"])["athletes_id"]})
    team["leaders"] = leaders

  if ("againstTheSpreadRecords" in team):
    team["againstTheSpreadRecords"] = unwrap_reference(team["againstTheSpreadRecords"])
    stats = {}
    for stat in team["againstTheSpreadRecords"]:
      stat_name = stat["type"]["name"]
      stats[stat_name] = stat
    team["againstTheSpreadRecords"] = stats

  if ("events" in team):
    team["events"] = unwrap_reference(team["events"])

  if ("transactions" in team):
    team["transactions"] = unwrap_reference(team["transactions"])

  if ("coaches" in team):
    team["coaches"] = unwrap_reference(team["coaches"])

  if ("groups" in team):
    team["groups"] = unwrap_reference(team["groups"])

  if ("ranks" in team):
    team["ranks"] = unwrap_reference(team["ranks"])

  if ("attendance" in team):
    team["attendance"] = unwrap_reference(team["attendance"])
    team["attendance"] = statistics_unwrap(team["attendance"]["categories"])

  team.pop("logos", None)
  team.pop("links", None)
  team.pop("alternateIds", None)
  team.pop("injuries", None)
  team.pop("notes", None)
  team.pop("franchise", None)
  if "venue" in team:
    team["venue"].pop("images", None)
  # json.dump(team, open(f"teams/team_{team['id']}.json", "w"))
  return team

url = "http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2021/teams/25?lang=en&region=us"
team = requests.get(url).json()
team = preprocess_team(team)

In [ ]:
team

In [ ]:
pd.set_option("display.max_columns", None)
pd.json_normalize(team)
# pd.json_normalize(game["competitors"], record_path="roster", record_prefix=".")

,$ref,id,guid,uid,slug,location,name,nickname,abbreviation,displayName,shortDisplayName,color,alternateColor,isActive,isAllStar,ranks,leaders,events,transactions,coaches,record.overall.$ref,record.overall.id,record.overall.name,record.overall.abbreviation,record.overall.type,record.overall.summary,record.overall.displayValue,record.overall.value,record.overall.OTLosses.name,record.overall.OTLosses.value,record.overall.OTWins.name,record.overall.OTWins.value,record.overall.avgPointsAgainst.name,record.overall.avgPointsAgainst.value,record.overall.avgPointsFor.name,record.overall.avgPointsFor.value,record.overall.clincher.name,record.overall.clincher.value,record.overall.differential.name,record.overall.differential.value,record.overall.divisionWinPercent.name,record.overall.divisionWinPercent.value,record.overall.gamesBehind.name,record.overall.gamesBehind.value,record.overall.gamesPlayed.name,record.overall.gamesPlayed.value,record.overall.leagueWinPercent.name,record.overall.leagueWinPercent.value,record.overall.losses.name,record.overall.losses.value,record.overall.playoffSeed.name,record.overall.playoffSeed.value,record.overall.pointDifferential.name,record.overall.pointDifferential.value,record.overall.points.name,record.overall.points.value,record.overall.pointsAgainst.name,record.overall.pointsAgainst.value,record.overall.pointsFor.name,record.overall.pointsFor.value,record.overall.streak.name,record.overall.streak.value,record.overall.ties.name,record.overall.ties.value,record.overall.winPercent.name,record.overall.winPercent.value,record.overall.wins.name,record.overall.wins.value,record.overall.divisionLosses.name,record.overall.divisionLosses.value,record.overall.divisionRecord.name,record.overall.divisionRecord.value,record.overall.divisionTies.name,record.overall.divisionTies.value,record.overall.divisionWins.name,record.overall.divisionWins.value,record.Home.$ref,record.Home.id,record.Home.name,record.Home.displayName,record.Home.shortDisplayName,record.Home.description,record.Home.type,record.Home.summary,record.Home.displayValue,record.Home.value,record.Home.wins.name,record.Home.wins.value,record.Home.losses.name,record.Home.losses.value,record.Home.ties.name,record.Home.ties.value,record.Home.winPercent.name,record.Home.winPercent.value,record.Home.OTLosses.name,record.Home.OTLosses.value,record.Road.$ref,record.Road.id,record.Road.name,record.Road.displayName,record.Road.shortDisplayName,record.Road.description,record.Road.type,record.Road.summary,record.Road.displayValue,record.Road.value,record.Road.wins.name,record.Road.wins.value,record.Road.losses.name,record.Road.losses.value,record.Road.ties.name,record.Road.ties.value,record.Road.winPercent.name,record.Road.winPercent.value,record.Road.OTLosses.name,record.Road.OTLosses.value,record.vs. Div..$ref,record.vs. Div..id,record.vs. Div..name,record.vs. Div..displayName,record.vs. Div..shortDisplayName,record.vs. Div..description,record.vs. Div..type,record.vs. Div..summary,record.vs. Div..displayValue,record.vs. Div..value,record.vs. Div..divisionWins.name,record.vs. Div..divisionWins.value,record.vs. Div..divisionLosses.name,record.vs. Div..divisionLosses.value,record.vs. Div..divisionTies.name,record.vs. Div..divisionTies.value,record.vs. Div..divisionWinPercent.name,record.vs. Div..divisionWinPercent.value,record.vs. Div..OTLosses.name,record.vs. Div..OTLosses.value,record.vs. Conf..$ref,record.vs. Conf..id,record.vs. Conf..name,record.vs. Conf..displayName,record.vs. Conf..shortDisplayName,record.vs. Conf..description,record.vs. Conf..type,record.vs. Conf..summary,record.vs. Conf..displayValue,record.vs. Conf..value,record.vs. Conf..wins.name,record.vs. Conf..wins.value,record.vs. Conf..losses.name,record.vs. Conf..losses.value,record.vs. Conf..ties.name,record.vs. Conf..ties.value,record.vs. Conf..leagueWinPercent.name,record.vs. Conf..leagueWinPercent.value,record.vs. Conf..OTLosses.name,record.vs. Conf..OTLosses.value,venue.$ref,venue.id,venue.fullNa

# Items list getter

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if (not os.path.exists("athlets_list.csv")):
  athlet_list = [i["$ref"] for i in unwrap_reference("https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes")]
  df = pd.DataFrame(athlet_list)
  df.to_csv("athlets_list.csv")

In [ ]:
if (not os.path.exists("teams_list.csv")):
  games_list = [i["$ref"] for i in unwrap_reference("https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{}/types/{}/weeks/{}/events")]
  games_list_df = pd.DataFrame(games_list)
  games_list_df.to_csv("teams_list.csv")

In [ ]:
if (not os.path.exists("events_list.csv")):
  games_list = []
  for season in range(2000, 2023):
    for week in range(40):
      for typ in range(5):
        try:
          games_list.extend([[season, typ, week, i["$ref"]] for i in unwrap_reference("https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{}/types/{}/weeks/{}/events".format(season, typ, week))])
        except:
          continue
  games_list_df = pd.DataFrame(games_list, columns =["season", "type", "week", "link"])
  games_list_df.to_csv("events_list.csv")

In [ ]:
print("Num of games: ", games_list_df.shape[0])

Num of games:  335


In [ ]:
def get_full_response_list(base):
    response = requests.get(base).json()
    response_list = []

    if 'pageIndex' in response.keys():
        for i in range(response['pageCount']):
            response_list.append(requests.get(base + f'&page={i + 1}').json()['items'])
        response_list = [x['$ref'] for x in list(chain.from_iterable(response_list))]
        # response_list_cl = list(chain.from_iterable(response_list))
    else:
        response_list = [x['$ref'] for x in response['items']]
        # response_list_cl = response['items']


    return response_list

In [ ]:
gamers_list = get_full_response_list('http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes?lang=en&region=us')

In [ ]:
isinstance(gamers_list[0], dict)

True

In [ ]:
from tqdm import tqdm
for i,j in tqdm(enumerate(gamers_list)):
  if isinstance(j, dict):
    continue
  gamers_list[i] = requests.get(j).json()

17973it [00:00, 1383755.38it/s]


In [ ]:
json.dump(gamers_list, open(f"/content/drive/MyDrive/Get All Teams/gamers_list.json", "w"))

In [ ]:
gamers_list[-1]['team']['$ref']

'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2023/teams/8?lang=en&region=us'

In [ ]:
from tqdm import tqdm
for i,j in tqdm(enumerate(gamers_list)):
  if isinstance(j, str):
    continue
  gamers_list[i] = j['$ref']

In [ ]:
teams_ref_list = set(gamers_list)

In [ ]:
teams_ref_list = list(teams_ref_list)

In [ ]:
json.dump(teams_ref_list, open(f"/content/drive/MyDrive/Get All Teams/teams_ref_list.json", "w"))

In [ ]:
teams_ref_list

In [ ]:
teams_ref_list[130]

'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2015/teams/28?lang=en&region=us'

In [ ]:
requests.get(teams_ref_list[130]).json().keys()

dict_keys(['$ref', 'id', 'guid', 'uid', 'alternateIds', 'slug', 'location', 'name', 'nickname', 'abbreviation', 'displayName', 'shortDisplayName', 'color', 'alternateColor', 'isActive', 'isAllStar', 'record', 'venue', 'groups', 'ranks', 'statistics', 'leaders', 'links', 'injuries', 'notes', 'againstTheSpreadRecords', 'franchise', 'events', 'transactions', 'coaches', 'attendance'])

In [ ]:
with open('/content/drive/MyDrive/Get All Teams/teams_ref_list.json', 'r', encoding='utf-8') as fh: #открываем файл на чтение
    teams_ref_list = json.load(fh) #загружаем из файла данные в словарь data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
teams_ref_list[130]

'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2015/teams/28?lang=en&region=us'

In [ ]:
# check_list = []
from tqdm import tqdm
for i, j in tqdm(enumerate(teams_ref_list[971:])):
  loaded_json = requests.get(j).json()

  # other loaded_json stats
  if ("details" in loaded_json):
    loaded_json["details"] = unwrap_reference(loaded_json["details"]) # loaded_json moments
  if ("leaders" in loaded_json):
    loaded_json["leaders"] = unwrap_reference(loaded_json["leaders"]) # loaded_json leaders by categroies
  if ("officials" in loaded_json):
    loaded_json["officials"] = unwrap_reference(loaded_json["officials"]) # ???
  if ("drives" in loaded_json):
    loaded_json["drives"] = unwrap_reference(loaded_json["drives"]) # some play events i dont know
  if ("record" in loaded_json):
    loaded_json["record"] = unwrap_reference(loaded_json["record"])
  if ("statistics" in loaded_json):
    loaded_json["statistics"] = unwrap_reference(loaded_json["statistics"])
  if ("injuries" in loaded_json):
    loaded_json["injuries"] = unwrap_reference(loaded_json["injuries"])
  if ("againstTheSpreadRecords" in loaded_json):
    loaded_json["againstTheSpreadRecords"] = unwrap_reference(loaded_json["againstTheSpreadRecords"])
  if ("events" in loaded_json):
    loaded_json["events"] = unwrap_reference(loaded_json["events"])
  if ("transactions" in loaded_json):
    loaded_json["transactions"] = unwrap_reference(loaded_json["transactions"])
  if ("coaches" in loaded_json):
    loaded_json["coaches"] = unwrap_reference(loaded_json["coaches"])
  if ("attendance" in loaded_json):
    loaded_json["attendance"] = unwrap_reference(loaded_json["attendance"])

  for p1, p2 in enumerate(loaded_json["injuries"]):
    loaded_json["injuries"][p1] = unwrap_reference(p2)

  if ("events" in loaded_json):
      for p1, p2 in enumerate(loaded_json["events"]):
        loaded_json["events"][p1] = unwrap_reference(p2)['id']
      loaded_json["events_ids"] = loaded_json["events"]

  for p1, p2 in enumerate(loaded_json["coaches"]):
    loaded_json["coaches"][p1] = unwrap_reference(p2['$ref'])

  if ('statistics' in loaded_json):
    if ('splits' in loaded_json['statistics']):
      loaded_json['statistics']['data'] = loaded_json['statistics']['splits'].get('categories', [])
    for g in [x for x in loaded_json['statistics'].keys() if x not in ('data', 'season', 'seasonType')]:
      del loaded_json['statistics'][g]

  bad_sections = set(["groups", "ranks", "links",\
                      "franchise", "logos", "isActive",\
                      "isAllStar", "events"])

  for section in list(loaded_json.keys()):
    if (section in bad_sections):
      del loaded_json[section]
    if (section.find("Available") != -1):
      del loaded_json[section]
    if (section.find("Source") != -1):
      del loaded_json[section]
  if ("venue" in loaded_json):
    loaded_json["venue"].pop("images", None)

  # check_list.append(loaded_json)
  tqdm.write('Processed '+ str(i+971) + ' bach')
  json.dump(loaded_json, open(f"/content/drive/MyDrive/Get All Teams/teams/team_number_"+str(i+972)+".json", "w"))

1it [00:13, 13.90s/it]

Processed 971 bach


1it [00:27, 13.90s/it]

Processed 972 bach


3it [00:43, 14.90s/it]

...

26it [06:58, 15.46s/it]

Processed 997 bach


27it [07:11, 15.09s/it]

Processed 998 bach


28it [07:11, 15.42s/it]
